In [174]:
import numpy as np

PATTERN_TO_DEBUG = 0

def toDec(hexstr,BIT):
    msb4bits = hexstr[0]
    n = int(msb4bits, 16)
    if n >= 8:
        p = -1*pow(2,BIT-1)
        addend = int(str(n-8) + hexstr[1:], 16)
        return str( p + addend)
    else:
        return str(int(hexstr, 16))

def int2hex(num,bit_width):
    if num<0:
        num = (1 << bit_width) + num
    hex_string = hex(num)[2:]
    return '{:0>{width}}'.format(hex_string,width = (bit_width+3)//4)

NEGATE = {'1': '0', '0': '1'}
def negate(value):
    return ''.join(NEGATE[x] for x in value)

def twocomplement(n, size_in_bits):
    # print(n)
    number = int(n)
    if number < 0:
        return negate(bin(abs(number) - 1)[2:]).rjust(size_in_bits, '1')
    else:
        return bin(number)[2:].rjust(size_in_bits, '0')

NEGATE = {'1': '0', '0': '1'}
def negate(value):
    return ''.join(NEGATE[x] for x in value)

def twocomplement(n, size_in_bits):
    # print(n)
    number = int(n)
    if number < 0:
        return negate(bin(abs(number) - 1)[2:]).rjust(size_in_bits, '1')
    else:
        return bin(number)[2:].rjust(size_in_bits, '0')


In [175]:
with open('input.txt', 'r') as file:
  file_in = file.readlines()

NUM = int(file_in[0])

# print(file_in)
for i in range(NUM):
  img_16 = []
  kernal_16 = []
  matrix_idx = []

  # MATRIX SIZE
  matrix_in_idx = int(file_in[1 + 50 * i + 0])

  if matrix_in_idx == 0:
     matrix_size = 8
  elif matrix_in_idx == 1:
     matrix_size = 16
  else:
     matrix_size = 32

  for j in range(16):
    img_in = np.array([int(toDec(val,8)) for val in file_in[1 + 50 * i + (1+j)].split()])
    img_16.append(np.reshape(img_in,(matrix_size,matrix_size)))

  for k in range(16):
    kernal_in = np.array([int(toDec(val,8)) for val in file_in[1 + 50 * i + (17+k)].split()])
    kernal_16.append(np.reshape(kernal_in,(5,5)))

  mode = int(file_in[1 + 50 * i + 33])

  for idx in range(16):
    matrix_idx.append(np.array([int(val) for val in file_in[1 + 50 * i + (34+idx)].split()]))

  if i == PATTERN_TO_DEBUG:
    break

In [176]:
img_16[0]

array([[2, 1, 1, 1, 1, 1, 1, 2],
       [1, 1, 1, 1, 2, 2, 2, 1],
       [1, 1, 1, 1, 2, 1, 1, 2],
       [1, 1, 1, 2, 1, 2, 2, 2],
       [2, 1, 1, 2, 1, 1, 1, 2],
       [1, 2, 2, 1, 1, 1, 2, 1],
       [1, 1, 1, 1, 2, 1, 1, 2],
       [2, 1, 2, 2, 1, 1, 2, 1]])

In [177]:
kernal_16[0]

array([[1, 1, 1, 1, 2],
       [2, 1, 2, 1, 1],
       [1, 1, 2, 1, 2],
       [2, 1, 2, 2, 1],
       [2, 2, 1, 2, 1]])

In [178]:
matrix_idx[0]

array([8, 0])

In [179]:
matrix_size

8

## Convolution

In [180]:
img = img_16[0]
kernal = kernal_16[0]

convoluted_results = np.zeros((matrix_size-4,matrix_size-4))
for i in range(matrix_size-4):
    for j in range(matrix_size-4):
        conv = 0
        for k in range(5):
            for l in range(5):
                conv += kernal[k,l] * img[i+k,j+l]

        convoluted_results[i,j] = conv

In [181]:
convoluted_results

array([[46., 43., 48., 55.],
       [47., 50., 50., 52.],
       [45., 47., 45., 53.],
       [49., 48., 51., 51.]])

## Maxpooling

In [182]:
convoluted_img_width = len(convoluted_results)

In [183]:
max_pooling_results = np.zeros((convoluted_img_width//2,convoluted_img_width//2))
for i in range(0,convoluted_img_width,2):
    for j in range(0,convoluted_img_width,2):
        max_pooling_results[i//2,j//2] = max(convoluted_results[i,j],convoluted_results[i+1,j],\
                                                convoluted_results[i+1,j+1],convoluted_results[i,j+1])

In [184]:
max_pooling_results

array([[50., 55.],
       [49., 53.]])

### Transposed convolution

In [185]:
trans_convoluted_results = np.zeros((matrix_size+4,matrix_size+4))
for i in range(matrix_size):
    for k in range(5):
        for j in range(matrix_size):
            for l in range(5):
                conv = kernal[k,l] * img[i,j]
                trans_convoluted_results[i+k,j+l] += conv

In [186]:
trans_convoluted_results[0]

array([2., 3., 4., 5., 8., 6., 6., 7., 6., 5., 4., 4.])

#### Convert results to LSB binary representation

In [187]:
twocomplement(-1,8)

'11111111'

In [188]:
vf = np.vectorize(twocomplement)
bits_result = vf(trans_convoluted_results,20)

In [189]:
bits_result[0][0]

'00000000000000000010'

In [190]:
result = bits_result[0][0]

In [191]:
result = result[::-1]

In [192]:
result

'01000000000000000000'